## Análisis de Simulated Annealing para una fuerza de ventas de 15 nodos

Simulated Annhealing es un algoritmo que recibe los siguientes hiperárametros de entrada: 

+ Tmax- Temperatura máxima con la que inicia el algoritmo
+ Tmin- Tempertatura mínima a la que llega al equilibrio
+ steps- Número de iteraciones
+ updates- Número de updates. 

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de simulated annhealing para un grafo con 15 nodos. Como primer objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 10 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 15 nodos. Se decidió elegir la fuerza de venta **96298** perteneciente a la Ciudad de México para llevar a cabo estas pruebas.

In [1]:
!pip install dynaconf
!pip install psycopg2-binary
!pip install simanneal

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the '/home/miguelmillan13/Documentos/Programas/anaconda3/bin/python -m pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the '/home/miguelmillan13/Documentos/Programas/anaconda3/bin/python -m pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the '/home/miguelmillan13/Documentos/Programas/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
# Librerías
import pandas as pd
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2

from src import Utileria as ut
from src.models import particle_swarm as ps
from src.models import simulated_annealing as sa

### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [2]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar: 5 nodos 80993
# 15 nodos 96298
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(96298))
df_Grafo

,id_origen,id_destino,distancia
0,11078,1009790047,3.1011127017081943
1,11078,1009791566,0.5142577124821974
2,11078,1020088646,0.9248048280652852
3,11078,1020249367,0.47811555481864304
4,11078,1020253076,0.6722824338378225
5,11078,1020300220,0.7785165511359913
6,11078,1020328100,0.9287372777889837
7,11078,1020449326,0.3605529558796124
8,11078,1020451581,0.46907337618604666
9,1009790047,1009791566,3.609300132764086


In [3]:
# Se crea el diccionario de hiper-parámetros que se evaluarán

#Default parameters
#Tmax = 25000.0  # Max (starting) temperature
#Tmin = 2.5      # Min (ending) temperature
#steps = 50000   # Number of iterations
#updates = 100   # Number of updates (by default an update prints to stdout)

#dict_Hiper_SA = {'Tmax': {1000,10000, 25000.0, 50000},
#              'Tmin': {1,2.5,5,10},
#              'steps': {500,5000,10000},
#              'updates': {10,50,100,200}
#              }
dict_Hiper_SA = {'Tmax': {10000, 25000},
              'Tmin': {1,2.5,5},
              'steps': {500,5000},
              'updates': {10,50,100}
              }


**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. En este caso se considerarán 3 valores de updates, 2 valores del steps, 2 valores de Tmax y 2 de Tmin dando lugar a un total de 36 combinaciones.. Cada combinación de hiperarámetros se correrá 10 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 10 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [4]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 10)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.20000          6.00    92.20%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.20000          6.00    91.77%     0.04%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.60%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.14%     0.00%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    93.10%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    91.63%     0.00%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.20000          6.00    91.80%     0.00%     0:00:00     0:00:00 Temperature 

CPU times: user 38min 54s, sys: 10.4 s, total: 39min 4s
Wall time: 48min 42s


In [5]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 100}",6,6,10/10
1,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 10}",6,6,10/10
2,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 5000, 'updates': 50}",6,6,10/10
3,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 100}",6,6,10/10
4,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 10}",6,6,10/10
5,"{'Tmax': 10000, 'Tmin': 1.0, 'steps': 500, 'updates': 50}",6,6,10/10
6,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 100}",6,6,10/10
7,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 10}",6,6,10/10
8,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 5000, 'updates': 50}",6,6,10/10
9,"{'Tmax': 10000, 'Tmin': 2.5, 'steps': 500, 'updates': 100}",6,6,10/10


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de ciertos de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones,se escogieron $10$ iteraciones para poder ver la variación entre iteraciones y la estabilidad del algoritmo. El número fue suficiente ya que en ninguna combinación de parámetros hubo variación en la distancia minima.

+ Sobre el resto de los parámetros, se escogieron valores alrededor e incluyendo los parámetros por omisión del algoritmo.

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió 10 veces, dando un total de 360 corridas. En las 10 corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima y máxima obtenida: $6 km$ es la misma para las 36 combinaciones. Esto demuestra que apesar de utilizar los mismos hiperparámetros, el algoritmo da los mismos resultados.

+ Estos resultados pueden ser consecuencia de haber utilizando los parámetros por default, por eso se hizo el siguiente experimento: correr el algoritmo 10 veces solo con 1 paso, updates con valor 2. Tmin con valor 1 y Tmax con valor 2. Fue interesante que la distancia minima y máxima de nuevo fue $6 km$. Puede que la estabilidad sea resultado del número pequeño de nodos, aunque corresponde al análisis realizado para 5 nodos.

In [11]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

dict_Hiper_SA = {'Tmax': {2},
              'Tmin': {1},
              'steps': {1},
              'updates': {2}
              }


df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 10)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.55%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.59%     0.47%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.20000          6.00    91.05%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.20000          6.00    92.89%     0.80%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.20000          6.00    92.70%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.20000          6.00    93.09%     0.83%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.90%     0.00%     0:00:00     0:00:00 Temperature 

CPU times: user 1min 6s, sys: 58.9 ms, total: 1min 6s
Wall time: 1min 6s


In [12]:
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Tmax': 2, 'Tmin': 1, 'steps': 1, 'updates': 2}",6,6,10/10


### 2. Análisis de rutas

In [6]:
rutas = pd.DataFrame(index=range(3),columns=['km', 'Ruta'])


dict_Hiper_SA = {'Tmax': 2,
              'Tmin': 1,
              'steps': 1,
              'updates': 2
              }

In [7]:
rutas

,km,Ruta
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN


In [9]:
for corrida in range(3):
    SA = sa.SimulatedAnnealing(df_Grafo,dict_Hiper_SA)
    SA.Ejecutar()
    
    min_distancia = round(SA.nbr_MejorCosto,3)
    mejor_ruta = SA.lst_MejorCamino
    
    rutas.km[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta
    
    

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    91.40%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.78%     0.61%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    93.10%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.82%     0.62%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.50%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.13000          6.00    92.62%     0.45%     0:00:06     0:00:00

In [10]:
rutas

,km,Ruta
0,6,"[1020451581, 1020449326, 1020249367, 1020300220, 1020088646, 11078, 1009791566, 1009790047, 1020..."
1,6,"[1009790047, 1020088646, 1020328100, 1020449326, 1020300220, 1020249367, 1020451581, 11078, 1020..."
2,6,"[1020300220, 1020328100, 1020451581, 1020253076, 1009790047, 1009791566, 1020088646, 1020249367,..."


### Conclusiones:
+ Los resultados del algoritmo pueden variar en cada corrida, aún cuando se mantengan fijos los hiperparámetros.
+ Este problema tiene múltiples minimos y regresa rutas distintas pero con el mismo mínimo, es decir la misma distancia mínima.


In [11]:
def convert(ruta): 
    s = [str(i) for i in ruta] 
    ruta_c = "-".join(s)
    return(ruta_c) 